# Slicing and indexing tutorial

In [1]:
## imports ##
# builtins
import os   # for os.chdir to navigate to directory where data is stored.
# external packages available via pip install 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# PlasmaCalcs package
import PlasmaCalcs as pc

from IPython.display import display, HTML  # set cell width in Jupyter:
display(HTML("<style>.container { width:90% !important; }</style>"))

# xarray options
xr.set_options(display_expand_data=False,
               keep_attrs=True, # -- keep attrs during operations. Note, doesn't handle conflicts, just takes attrs from the first array.
              );

In [2]:
## create an EppicCalculator object, from example data (in tests/test_eppic/test_eppic_tinybox)##
EPPIC_TINYBOX_DIR = pc.pc_path('tests', 'test_eppic', 'test_eppic_tinybox')
with pc.InDir(EPPIC_TINYBOX_DIR):  # temporarily cd into that directory ^
    ec = pc.EppicCalculator.from_here(u_t=1, kw_units=dict(M=1))  # u_t=1 & M=1 <--> SI units for time & mass

ec

EppicCalculator(input_deck=EppicInputDeck(with 3 distributions, filename='/Users/tessgoodman/codepackages/plasmacalcs/tests/test_eppic/test_eppic_tinybox/eppic.i'), dims=DimRegion(snap=SnapList(len=10; Snap('2560', 0), ..., Snap('25600', 9)), fluid=EppicDistList(EppicDist('e-', 0), EppicDist('H+', 1), EppicDist('C+', 2)), jfluid=EppicNeutral('neutral', 0), component=ComponentList(Component('x', 0), Component('y', 1), Component('z', 2))))

In [3]:
#snap, component, fluid, and jfluid are special dimensions in plasmacalcs
#they each have a "current value" and a list of all possible values
#for example, here is the current snap value:
ec.snap

SnapList(len=10; Snap('2560', 0, t=3.840e-05), ..., Snap('25600', 9, t=3.840e-04))

In [4]:
#you can alter the current value and store it back in ec.snap like this:
ec.snap=1

In [5]:
#now, when you call the current value, only the first index of the snap object shows
ec.snap

Snap('5120', 1, t=7.680e-05)

In [6]:
#if you want to reset ec.snap:
ec.snap = None
ec.snap

SnapList(len=10; Snap('2560', 0, t=3.840e-05), ..., Snap('25600', 9, t=3.840e-04))

In [7]:
#to get the full list of snap values, use:
ec.snaps

SnapList(len=10; Snap('2560', 0, t=3.840e-05), ..., Snap('25600', 9, t=3.840e-04))

In [8]:
#to load an ec object at every fourth snapshot:
ec('E', snap = slice(None, None, 4))

#note: this is more efficient than using ec('E').isel(snap = slice(None, None, 4)) because it only loads the sliced snapshots

<xarray.DataArray 'E' (component: 3, snap: 3, x: 16, y: 16)> Size: 18kB
2.612 2.045 2.512 2.977 3.216 2.348 1.769 2.038 ... 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * component  (component) object 24B x y z
  * x          (x) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * y          (y) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * snap       (snap) object 24B 2560 12800 23040
    t          (snap) float64 24B 3.84e-05 0.000192 0.0003456
Attributes:
    units:    si

In [9]:
#here is another way to do the same thing:
original_snap = ec.snap

In [10]:
#this alters the current value of snap
ec.snap = slice(None, None, 4)

In [11]:
#notice how the snap object now houses only every fourth snapshot
result = ec('E')
ec('E')

<xarray.DataArray 'E' (component: 3, snap: 3, x: 16, y: 16)> Size: 18kB
2.612 2.045 2.512 2.977 3.216 2.348 1.769 2.038 ... 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * component  (component) object 24B x y z
  * x          (x) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * y          (y) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * snap       (snap) object 24B 2560 12800 23040
    t          (snap) float64 24B 3.84e-05 0.000192 0.0003456
Attributes:
    units:    si

In [12]:
#reset the original values
ec.snap = original_snap

In [13]:
#check to see that all snaps are back
ec('E')

<xarray.DataArray 'E' (component: 3, snap: 10, x: 16, y: 16)> Size: 61kB
2.612 2.045 2.512 2.977 3.216 2.348 1.769 2.038 ... 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * component  (component) object 24B x y z
  * x          (x) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * y          (y) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * snap       (snap) object 80B 2560 5120 7680 10240 ... 20480 23040 25600
    t          (snap) float64 80B 3.84e-05 7.68e-05 ... 0.0003456 0.000384
Attributes:
    units:    si

### setting values for the special dimensions

In [14]:
#you can use the string name of a special dimension to set its value:
ec.fluid = 'e-'
ec.fluid

EppicDist(N=0, name='e-', m=9.11e-31, q=-1.60e-19, n0=3.60e+11)

In [15]:
ec.fluid = [0, 'H+']
ec.fluid

[EppicDist(N=0, name='e-', m=9.11e-31, q=-1.60e-19, n0=3.60e+11),
 EppicDist(N=1, name='H+', m=1.67e-27, q=1.60e-19, n0=3.00e+11)]

In [16]:
#you can index to get specific snaps like this
ec.snap = [2, 5]
ec.snap

[Snap('7680', 2, t=1.152e-04), Snap('15360', 5, t=2.304e-04)]

In [17]:
#>>>you can use fractional indexing to slice objects with given percentages<<<
ec.snap = slice(0.1, -0.3, 0.20)
#this sets snaps to start 10% of the way through its original list and ending 30% from the end, taking steps of 20%
#(good for data exploration, but internal rounding might make it hard to control exactly what happens;
#  for precise control just use regular slices, e.g. ec.snap = slice(5, 15, 3).)
list(ec.snap)

[Snap('5120', 1, t=7.680e-05),
 Snap('10240', 3, t=1.536e-04),
 Snap('15360', 5, t=2.304e-04),
 Snap('20480', 7, t=3.072e-04)]

In [18]:
#recall: to reset all snap values use None
ec.snap = None

#check that all 10 snap objects are there:
ec('E').snap

<xarray.DataArray 'snap' (snap: 10)> Size: 80B
2560 5120 7680 10240 12800 15360 17920 20480 23040 25600
Coordinates:
  * snap     (snap) object 80B 2560 5120 7680 10240 ... 17920 20480 23040 25600
    t        (snap) float64 80B 3.84e-05 7.68e-05 ... 0.0003456 0.000384

In [19]:
#in some cases, there are special options
#fluid has these selectors: ELECTRON, ELECTRONS, ION, IONS, CHARGED, NEUTRAL, NEUTRALS
ec.fluid = pc.ELECTRON
ec.fluid
#ELECTRON works no matter what the string name of your electron is ('e', 'e-', etc.) so it is useful in a generic sense

EppicDist(N=0, name='e-', m=9.11e-31, q=-1.60e-19, n0=3.60e+11)

In [20]:
#another example:
ec.fluid = pc.IONS
ec.fluid
#note that pc.ION will not work in this case because the singular only works if there is 1 of that kind of fluid

EppicDistList(EppicDist(N=1, name='H+', m=1.67e-27, q=1.60e-19, n0=3.00e+11), EppicDist(N=2, name='C+', m=1.99e-26, q=1.60e-19, n0=6.00e+10))

In [21]:
#snap has these selectors: SELECT_CLOSEST, SELECT_AFTER, SELECT_BEFORE, SELECT_ALL_AFTER, SELECT_ALL_BEFORE, SELECT_BETWEEN
ec.snap = pc.SELECT_BETWEEN(0.0003, 0.0007) #this selects all snaps between t=0.0003 and t=0.0007
ec.snap

SnapList(Snap('20480', 7, t=3.072e-04), Snap('23040', 8, t=3.456e-04), Snap('25600', 9, t=3.840e-04))

In [22]:
ec.snap = pc.SELECT_CLOSEST(0.0001) #this selects the snap closest to t=0.0001
ec.snap

Snap('7680', 2, t=1.152e-04)

In [23]:
ec.snap = pc.SELECT_ALL_AFTER(0.00019) #this selects all snaps after t=0.00019
ec.snap

SnapList(len=6; Snap('12800', 4, t=1.920e-04), ..., Snap('25600', 9, t=3.840e-04))

In [24]:
#you can also use the xarray .isel method to select certain indexes
E = ec('E')
E_sliced = E.isel(component=[0,1], snap=slice(None, None, 4))
E_sliced

<xarray.DataArray 'E' (component: 2, snap: 2, x: 16, y: 16)> Size: 8kB
2.578 2.316 3.807 4.24 2.72 1.946 ... -8.274 -8.393 -8.056 -7.818 -8.519 -9.091
Coordinates:
  * component  (component) object 16B x y
  * x          (x) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * y          (y) float64 128B 0.0 0.022 0.044 0.066 ... 0.264 0.286 0.308 0.33
  * snap       (snap) object 16B 12800 23040
    t          (snap) float64 16B 0.000192 0.0003456
Attributes:
    units:    si

In [25]:
#if you want to use plasmacalcs special functions in conjunction with isel, you must use pc.isel.
#for example, use this to apply fractional indexing to an already-loaded array:
ec.snap = None
E = ec('E')
E.pc.isel(snap=slice(None, 0.4)).snap  #selects the first 40% of snaps from E

<xarray.DataArray 'snap' (snap: 4)> Size: 32B
2560 5120 7680 10240
Coordinates:
  * snap     (snap) object 32B 2560 5120 7680 10240
    t        (snap) float64 32B 3.84e-05 7.68e-05 0.0001152 0.0001536

In [26]:
#or, another example using pc.isel, here to apply a special selector to an already-loaded array:
ec.fluid = None
n = ec('n')
print('before .pc.isel:', n.fluid.values)   # <-- all the fluids in ec.fluids
print('after .pc.isel:', n.pc.isel(fluid=pc.IONS).fluid.values)  # <-- only the ions

before .pc.isel: [EppicDist(N=0, name='e-', m=9.11e-31, q=-1.60e-19, n0=3.60e+11)
 EppicDist(N=1, name='H+', m=1.67e-27, q=1.60e-19, n0=3.00e+11)
 EppicDist(N=2, name='C+', m=1.99e-26, q=1.60e-19, n0=6.00e+10)]
after .pc.isel: [EppicDist(N=1, name='H+', m=1.67e-27, q=1.60e-19, n0=3.00e+11)
 EppicDist(N=2, name='C+', m=1.99e-26, q=1.60e-19, n0=6.00e+10)]
